---

# Training Auto-encoder

>This notebook contains code for training the auto-encoder with reconstruction loss.
>Audio is cropped to 1 second (44100 samples). ie. the model generates 1 second of audio for each inference.

---

In [1]:
import pandas as pd
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import IPython.display as ipd

# Dataset

### Load audio file path-geotag mappings

In [2]:
df = pd.read_csv('../corpus/mappings.csv')

In [3]:
df.head()

,path,lat,long
0,../corpus/audio/169884.mp3,0.501047,0.041804
1,../corpus/audio/169885.mp3,0.501052,0.041806
2,../corpus/audio/697381.mp3,0.187898,0.534158
3,../corpus/audio/187893.mp3,0.459852,0.012109
4,../corpus/audio/788102.mp3,0.236748,-0.876991


### Load audio files
- Crop audio to 44100 samples.
- Pad audio to 44100 samples.

In [4]:
audio = list()

for i, row in df.iterrows():
    y, sr = librosa.load(row['path'])

    y = np.pad(y, (0, max(0, 44100 - len(y))), mode="constant")[:44100]

    audio.append(y)

In [5]:
y = np.array(audio)
y.shape

(105, 44100)

## Auto-encoder

### Encoder

In [6]:
# audio_shape = (44100,) for each audio sample
audio_shape = (44100,)

audio_input = layers.Input(shape=audio_shape)

x = layers.Reshape((audio_shape[0], 1))(audio_input)
x = layers.Conv1D(32, 3, activation='relu')(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(64, 3, activation='relu')(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='tanh')(x)

encoder = models.Model(audio_input, x)

### Decoder

In [7]:
latent_input = layers.Input(shape=(64,))
x = layers.Dense(128, activation='relu')(latent_input)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(44100, activation='tanh')(x)
x = layers.Reshape((44100,))(x)

decoder = models.Model(latent_input, x)

### Combine encoder and decoder

In [8]:
encoding = encoder(audio_input)
decoder_output = decoder(encoding)

ae = models.Model(audio_input, decoder_output)

In [9]:
ae.compile(optimizer='adam', loss='mean_squared_error')
ae.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 44100)]           0         
                                                                 
 model (Functional)          (None, 64)                45156608  
                                                                 
 model_1 (Functional)        (None, 44100)             11375044  
                                                                 
Total params: 56,531,652
Trainable params: 56,531,652
Non-trainable params: 0
_________________________________________________________________


### Train model

Train the auto-encoder with reconstruction loss

In [26]:
ae.fit(y, y, epochs=50)

Epoch 1/50
4/4 [==============================] - 4s 926ms/step - loss: 5.1073e-06
Epoch 2/50
4/4 [==============================] - 4s 899ms/step - loss: 4.4230e-06
Epoch 3/50
4/4 [==============================] - 4s 897ms/step - loss: 4.3527e-06
Epoch 4/50
4/4 [==============================] - 4s 952ms/step - loss: 4.4569e-06
Epoch 5/50
4/4 [==============================] - 4s 914ms/step - loss: 5.7927e-06
Epoch 6/50
4/4 [==============================] - 4s 937ms/step - loss: 8.0166e-06
Epoch 7/50
4/4 [==============================] - 4s 935ms/step - loss: 8.1334e-06
Epoch 8/50
4/4 [==============================] - 4s 900ms/step - loss: 6.6504e-06
Epoch 9/50
4/4 [==============================] - 4s 910ms/step - loss: 5.6943e-06
Epoch 10/50
4/4 [==============================] - 4s 895ms/step - loss: 5.7526e-06
Epoch 11/50
4/4 [==============================] - 4s 926ms/step - loss: 5.3551e-06
Epoch 12/50
4/4 [==============================] - 4s 887ms/step - loss: 5.2093e-06
E

### Test model

In [27]:
y_test = y[0]

ipd.display(ipd.Audio(data=y_test, rate=44100))

In [28]:
example_vector = encoder.predict(np.array([y_test]))

y_recon = decoder.predict(example_vector, verbose=False)[0]

1/1 [==============================] - 0s 31ms/step


In [30]:
ipd.display(ipd.Audio(data=y_recon, rate=44100))

### Save models

In [31]:
encoder.save('../models/encoder.h5')
decoder.save('../models/decoder.h5')